# 01 spaCy Exploration

In [1]:
import os
from pathlib import Path

import imgkit
import pandas as pd
import spacy
from IPython.core.display import HTML, display
from IPython.display import Image
from spacy import displacy

In [2]:
pd.set_option(
    "display.colheader_justify", "left"
)  # sets the default alignment of column headers to 'left'
display(HTML("<style>.container { width:100% !important; }</style>"))

### Helper Functions

In [3]:
def show(table):
    with pd.option_context("display.max_colwidth", None):
        display(table)

### Starting with spaCy

In [4]:
# Prints version of spaCy in use
print(spacy.__version__)

2.3.2


The primary language model will be the larger model `de_core_news_lg`. To point out the differences, the small model `de_core_news_sm` is loaded, too.

In [5]:
gerNLPsm = spacy.load("de_core_news_sm")
gerNLPlg = spacy.load("de_core_news_lg")

In [6]:
# Calling 'spacy.info()' on the German model returns the model's meta data.
info = spacy.info("de_core_news_lg")


===================== Info about model 'de_core_news_lg' =====================

lang             de                            
name             core_news_lg                  
license          MIT                           
author           Explosion                     
url              https://explosion.ai          
email            contact@explosion.ai          
sources          [{'name': 'TIGER Corpus', 'url': 'https://www.ims.uni-stuttgart.de/forschung/ressourcen/korpora/tiger.html', 'license': 'commercial (licensed by Explosion)'}, {'name': 'WikiNER', 'url': 'https://figshare.com/articles/Learning_multilingual_named_entity_recognition_from_Wikipedia/5462500', 'license': 'CC BY 4.0'}, {'name': 'OSCAR (Common Crawl)', 'url': 'https://oscar-corpus.com/', 'license': 'CC0'}, {'name': 'Wikipedia (20200201)', 'url': 'https://dumps.wikimedia.org/', 'license': 'CC BY-SA 3.0'}]
description      German multi-task CNN trained on the TIGER and WikiNER corpora. Assigns word vectors, POS tags,

# Linguistic Features in Speech Parsing

For the exploration of the configuration and features of the library spaCy, a speech given on March 23rd 2011 during the parliament's question time by Ursula Heinen-Esser (CDU) is loaded as a document. At the time, Ursula Heinen-Esser was parliamentary undersecretary to the Federal Minister for the Environment, Nature Conservation and Nuclear Safety.

To get started, the raw text is converted to a doc object using the German language model previously loaded. As the doc object is created, tokenization is done, too.

In [7]:
doc_txt = open("data/sample_speech.txt", "r", encoding="utf8")
doc = gerNLPlg(doc_txt.read())

with pd.option_context("display.max_colwidth", 25):
    print(doc)

Frau Präsidentin! Sehr geehrte Damen und Herren! Liebe Kolleginnen und Kollegen! Ich glaube, es isl wirklich unzweifelhaft: Die nuklearen Folgen der Erdbebenkatastrophe in Japan bedeuten einen Einschnitt, zuallererst selbstverständlich für Japan, aber auch für die ganze Welt. Die Katastrophe hat ganz deutlich gezeigt, dass Ereignisse auch jenseits der bislang berücksichtigten Szenarien eintreten können. Vielleicht noch ein paar Punkte zum Sachverhalt, weil es im Weiteren darum gehen wird - so verstehe ich das Thema der Aktuellen Stunde -, welche Sicherheitsüberprüfungen es in unseren deutschen Kernkraftwerken geben wird. Bei allen betroffenen Reaktoren gab es ein Zusammentreffen eines extremen Erdbebens mit einem Tsunami. Das Zusammenwirken hat zum Ausfall der externen Stromversorgung geführt. In der Folge wurden die notwendigen Sicherheitseinrichtungen zerstört. Die Kernkühlung bei den Blöcken 1 bis 3 am Standort Fukushima fiel aus. Die Blöcke 4 bis 6 waren zu diesem Zeitpunkt abgesch

In [8]:
sentences = [sentence.orth_ for sentence in doc.sents]
words = [token.orth_ for token in doc if token.pos_ != "PUNCT"]
print(
    "The sample speech contains "
    + str(len(sentences))
    + " sentences and "
    + str(len(words))
    + " words in total."
)

The sample speech contains 60 sentences and 1019 words in total.


## Part-of-Speech (POS) tagging
The next chunk of code returns a table of tokens 289 to 317, a sample sentence of the speech, with labels. spaCy tokenizes everything (words, numbers, punctuation, etc.) except single spaces next to words. The POS label indicate language-universal syntactic token positions, the TAG label indicates position labels specifically for the German language using the STTS. https://explosion.ai/blog/german-model#Data-sources

In [9]:
pos_tags = []  # empty list to be filled later
# for every token the loop appends the tags and labels to the list
for token in doc:
    pos_tags.append(
        (token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.is_stop)
    )

# converts the list to a dataframe which can be visualized nicely in markdown
pos_tags = pd.DataFrame(
    pos_tags, columns=("TEXT", "LEMMA", "POS", "TAG", "DEP", "STOP")
)
show(pos_tags[289:317])

,TEXT,LEMMA,POS,TAG,DEP,STOP
289,Die,der,DET,ART,nk,True
290,Ereignisse,Ereignis,NOUN,NN,sb,False
291,in,in,ADP,APPR,mnr,True
292,Japan,Japan,PROPN,NE,nk,False
293,haben,haben,AUX,VAFIN,ROOT,True
294,uns,sich,PRON,PPER,da,True
295,gezeigt,zeigen,VERB,VVPP,oc,False
296,",",",",PUNCT,"$,",punct,False
297,dass,dass,SCONJ,KOUS,cp,True
298,das,der,DET,ART,nk,True


### Decoding the tag labels
Calling *spacy.explain()* on the TAG or POS will return a more profound explanation of the tag closer to human understanding of language.

In [10]:
tags_explained = []
for token in doc:
    tags_explained.append(
        (
            token.text,
            token.pos_,
            spacy.explain(token.pos_),
            token.tag_,
            spacy.explain(token.tag_),
        )
    )

tags_explained = pd.DataFrame(
    tags_explained, columns=("TEXT", "POS", "POS_Explained", "TAG", "TAG_Explained")
)
show(tags_explained[289:317])

,TEXT,POS,POS_Explained,TAG,TAG_Explained
289,Die,DET,determiner,ART,definite or indefinite article
290,Ereignisse,NOUN,noun,NN,"noun, singular or mass"
291,in,ADP,adposition,APPR,preposition; circumposition left
292,Japan,PROPN,proper noun,NE,proper noun
293,haben,AUX,auxiliary,VAFIN,"finite verb, auxiliary"
294,uns,PRON,pronoun,PPER,non-reflexive personal pronoun
295,gezeigt,VERB,verb,VVPP,"perfect participle, full"
296,",",PUNCT,punctuation,"$,",comma
297,dass,SCONJ,subordinating conjunction,KOUS,subordinate conjunction with sentence
298,das,DET,determiner,ART,definite or indefinite article


In the sample speech of 65 sentences and 1019 words in total there are 40 unique tags, each shown with an example in the 'TEXT' columns in the next table.
\label{se-unique-tags}

In [11]:
unique_tags = tags_explained.drop_duplicates("TAG_Explained").sort_values(by=["TAG"])[
    ["TAG", "TAG_Explained", "POS", "POS_Explained", "TEXT"]
]
unique_tags = pd.DataFrame(
    unique_tags, columns=("TAG", "TAG_Explained", "POS", "POS_Explained", "TEXT")
).reset_index()
print(f"Table shaped {str(unique_tags.shape)} with unique values")
show(
    pd.DataFrame(
        unique_tags, columns=("TAG", "TAG_Explained", "POS", "POS_Explained", "TEXT")
    )
)

Table shaped (41, 6) with unique values


,TAG,TAG_Explained,POS,POS_Explained,TEXT
0,$(,other sentence-internal punctuation mark,PUNCT,punctuation,-
1,"$,",comma,PUNCT,punctuation,","
2,$.,sentence-final punctuation mark,PUNCT,punctuation,!
3,ADJA,"adjective, attributive",ADJ,adjective,geehrte
4,ADJD,"adjective, adverbial or predicative",ADJ,adjective,wirklich
5,ADV,adverb,ADV,adverb,Sehr
6,APPR,preposition; circumposition left,ADP,adposition,in
7,APPRART,preposition with article,ADP,adposition,zum
8,ART,definite or indefinite article,DET,determiner,Die
9,CARD,cardinal number,NUM,numeral,1


## Dependencies
The German language exhibits a special morpological characteristic in comparison to English. German is non-projective and generally less restrictive, which allows for various word orders in a sentence without changing the meaning. It's recognizable by crossing dependency arcs. See this spaCy blog post on [word order](https://explosion.ai/blog/german-model#word-order) and this introcution to the [TIGER annotation treebank](https://www.ims.uni-stuttgart.de/documents/ressourcen/korpora/tiger-corpus/annotation/tiger_introduction.pdf) for more detail on German linguistics for NLP.

With Displacy, another tool from the same creators of spaCy, visualizes parsed text with POS tags and the dependency tags between related tokens.

In [12]:
displacy_options = {
    "fine_grained": False,
    "add_lemma": True,
    "collapse_phrases": False,
    "compact": True,
    "arrow_stroke": 1,
    "distance": 100,
}


def displacy_visual(
    spaCy_doc, style="dep", options=displacy_options, minify=True, jupyter=True
):
    """Takes a spaCy document and returns a visual of the dependency parse."""
    visual = displacy.render(spaCy_doc, style, options=options)
    return visual

In [13]:
example_text = "Ich glaube, wir sollten bei dieser Branche einen Schwerpunkt setzen, da sie uns weg von \
Atomenergie und fossilen Energieträgern hin zu dezentralen Lösungen führt, und nicht schon jetzt Kürzungen \
vornehmen, obwohl die Branche noch nicht einmal richtig etabliert ist."
example = gerNLPlg(example_text)
example_sm = gerNLPsm(example_text)

### Parsed dependencies with the small model versus the large model
Admittedly, the example at hand is a long, convoluted sentence. But it's a real-world example being part of the full document described above.

The example starts with a subclause kicking-off indirect speech _"Ich glaube"_, followed by a main clause _"wir sollten bei dieser Branche einen Schwerpunkt setzen und nicht schon jetzt Kürzungen vornehmen"_ with an embedded subclause _"da sie uns weg von Atomenergie und fossilen Energieträgern hin zu dezentralen Lösungen führt"_, and rounded-off by another subclause _"obwohl die Branche noch nicht einmal richtig etabliert ist."_. The last clause is a subclause due to the subordinating conjunction `SCONJ`. The full example with explained dependency and POS tags follows later in the notebook.

The first visual uses the small model, the second uses the large model. There is only minor difference to be spotted:

- Parsed with the small model, "führt" is connected with "und" as a coordinating conjunction (edge tag `cd`)
- Parsed with the large model, "und" is no longer connected with "führt", but with "setzen" in the first main clause as a coordinating conjunction

The small model identifies the second part of the main clause to be a coordinating conjunction to the embedded subclause, though coordinating conjunctions only connect two main clauses. The large model connects the second part of the main clause with the first, which is right.

This deviation into the differences between the language models only touches the surface. I'll keep using the larger model from now.

In [14]:
displacy_visual(example_sm)

### Parsed dependencies with large model

In [15]:
displacy_visual(example)

In [16]:
short_doc = gerNLPlg(
    "Die Ereignisse in Japan haben uns gezeigt, dass das sogenannte Restrisiko durchaus existent ist \
und dass es sich hierbei nicht nur um eine rechnerische Größe handelt."
)

In [17]:
displacy_visual(short_doc)

### Investigating the parse tree
The arcs shown in the dependency parsing visualization with displacy define the syntactic relation between two words. The arcs are directional, which indicates the status of head and child inside of the dependency tree. In the following chunk, the entire dependency tree is parsed.

In [18]:
parse_tree = []
for token in example:
    parse_tree.append(
        (
            token.text,
            token.pos_,
            spacy.explain(token.pos_),
            token.dep_,
            spacy.explain(token.dep_),
            token.head.text,
            token.head.pos_,
            [child for child in token.children],
        )
    )

parse_tree = pd.DataFrame(
    parse_tree,
    columns=(
        "TOKEN_TEXT",
        "TEXT_POS",
        "POS_Explained",
        "DEP",
        "DEP_Explained",
        "HEAD_TEXT",
        "HEAD_POS",
        "CHILDREN",
    ),
)
show(parse_tree)

,TOKEN_TEXT,TEXT_POS,POS_Explained,DEP,DEP_Explained,HEAD_TEXT,HEAD_POS,CHILDREN
0,Ich,PRON,pronoun,sb,subject,glaube,VERB,[]
1,glaube,VERB,verb,ROOT,None,glaube,VERB,"[Ich, ,, sollten, .]"
2,",",PUNCT,punctuation,punct,punctuation,glaube,VERB,[]
3,wir,PRON,pronoun,sb,subject,sollten,VERB,[]
4,sollten,VERB,verb,oc,clausal object,glaube,VERB,"[wir, setzen, ,, und]"
5,bei,ADP,adposition,mo,modifier,setzen,VERB,[Branche]
6,dieser,DET,determiner,nk,noun kernel element,Branche,NOUN,[]
7,Branche,NOUN,noun,nk,noun kernel element,bei,ADP,[dieser]
8,einen,DET,determiner,nk,noun kernel element,Schwerpunkt,NOUN,[]
9,Schwerpunkt,NOUN,noun,oa,accusative object,setzen,VERB,[einen]


In [19]:
parse_tree = []
for token in doc:
    parse_tree.append(
        (
            token.text,
            token.pos_,
            spacy.explain(token.pos_),
            token.dep_,
            spacy.explain(token.dep_),
            token.head.text,
            token.head.pos_,
            [child for child in token.children],
        )
    )

parse_tree = pd.DataFrame(
    parse_tree,
    columns=(
        "TOKEN_TEXT",
        "TEXT_POS",
        "POS_Explained",
        "DEP",
        "DEP_Explained",
        "HEAD_TEXT",
        "HEAD_POS",
        "CHILDREN",
    ),
)
show(parse_tree[289:317])

,TOKEN_TEXT,TEXT_POS,POS_Explained,DEP,DEP_Explained,HEAD_TEXT,HEAD_POS,CHILDREN
289,Die,DET,determiner,nk,noun kernel element,Ereignisse,NOUN,[]
290,Ereignisse,NOUN,noun,sb,subject,haben,AUX,"[Die, in]"
291,in,ADP,adposition,mnr,postnominal modifier,Ereignisse,NOUN,[Japan]
292,Japan,PROPN,proper noun,nk,noun kernel element,in,ADP,[]
293,haben,AUX,auxiliary,ROOT,None,haben,AUX,"[Ereignisse, gezeigt, .]"
294,uns,PRON,pronoun,da,dative,gezeigt,VERB,[]
295,gezeigt,VERB,verb,oc,clausal object,haben,AUX,"[uns, ,, ist]"
296,",",PUNCT,punctuation,punct,punctuation,gezeigt,VERB,[]
297,dass,SCONJ,subordinating conjunction,cp,complementizer,ist,AUX,[]
298,das,DET,determiner,nk,noun kernel element,Restrisiko,NOUN,[]


The parse tree above shows that dependency tags are still accessed via tokens, despite actually explaining what the relationship between two tokens looks like. The dependency between a token and its head can be accessed via the token. As the sentence root does not have a head, no dependency can be accessed via the root.

In [20]:
unique_dep = parse_tree.drop_duplicates("DEP").sort_values(by=["DEP"])[
    ["DEP", "DEP_Explained"]
]
show(unique_dep)

,DEP,DEP_Explained
879,,None
0,ROOT,None
201,ac,adpositional case marker
26,ag,genitive attribute
627,ams,measure argument of adjective
752,app,apposition
687,cc,coordinating conjunction
6,cd,coordinating conjunction
7,cj,conjunct
686,cm,comparative conjunction


Viewing the table it becomes clear that first degree relations can proove to be meaningful. For example, 'Restrisiko' (remaining risk) is the direct head to the adjective 'sogenannte' (so-called) which carries judging of the noun 'Restrisiko'.
Nevertheless, working only with first degree relationships would disregard plenty of information hidden in the sentence. For example, the adjective 'durchaus' (indeed) which describes the adverb 'existent' (existant) is related to 'Restrisiko' through the auxiliary verb 'ist' (is). This valuable information is lost if the dependency tree is not crawled through in order to investigate local trees.

### Crawling through the local tree
Previously we have only investigated single archs. Looking at local trees will put our focus on a per sentence level. The ultimate interest is to iterate over the token to find chains, in other words to follow an arch to another arch to another arch. Later, in the analysis, conditions for the continuation of the iteration can be set, such as the POS/TAG label having to be a verb and then an adverb since we are looking at such discriptive words.

#### Local Surrounding per Token
It is of interest to follow along the word hierarchy. Therefore, let's first look at the local surroundings of each token in the next table. Syntactic children are words which are connected by an arch to the token, the distinction in left and right refers to their appearance before or after the token. This number gives an idea about the surrounding of each token. The token head, which is also returned, will indicate the location of the root - it is where both token and token head are similar.

In [21]:
local_trees = []
for token in doc:
    local_trees.append(
        (
            token.text,
            token.head,
            token.n_lefts + token.n_rights,
            token.n_lefts,
            token.n_rights,
        )
    )

local_trees = pd.DataFrame(
    local_trees,
    columns=("TOKEN_TEXT", "TOKEN_HEAD", "TOTAL_CHILD", "LEFT_CHILD", "RIGHT_CHILD"),
)
show(local_trees[289:317])

,TOKEN_TEXT,TOKEN_HEAD,TOTAL_CHILD,LEFT_CHILD,RIGHT_CHILD
289,Die,Ereignisse,0,0,0
290,Ereignisse,haben,2,1,1
291,in,Ereignisse,1,0,1
292,Japan,in,0,0,0
293,haben,haben,3,1,2
294,uns,gezeigt,0,0,0
295,gezeigt,haben,3,1,2
296,",",gezeigt,0,0,0
297,dass,ist,0,0,0
298,das,Restrisiko,0,0,0


#### Descendants per Token
This looks at the subtree of the token and returns any words the token archs out to. Thus, also words beyond the first degree are included.

In [22]:
descendants = []
for token in example:
    descendants.append(
        (
            token.text,
            [descendant.text for descendant in token.subtree if token != descendant],
        )
    )

descendants = pd.DataFrame(descendants, columns=("TOKEN_TEXT", "DESCENDANTS"))
show(descendants)

,TOKEN_TEXT,DESCENDANTS
0,Ich,[]
1,glaube,"[Ich, ,, wir, sollten, bei, dieser, Branche, einen, Schwerpunkt, setzen, ,, da, sie, uns, weg, von, Atomenergie, und, fossilen, Energieträgern, hin, zu, dezentralen, Lösungen, führt, ,, und, nicht, schon, jetzt, Kürzungen, vornehmen, ,, obwohl, die, Branche, noch, nicht, einmal, richtig, etabliert, ist, .]"
2,",",[]
3,wir,[]
4,sollten,"[wir, bei, dieser, Branche, einen, Schwerpunkt, setzen, ,, da, sie, uns, weg, von, Atomenergie, und, fossilen, Energieträgern, hin, zu, dezentralen, Lösungen, führt, ,, und, nicht, schon, jetzt, Kürzungen, vornehmen, ,, obwohl, die, Branche, noch, nicht, einmal, richtig, etabliert, ist]"
5,bei,"[dieser, Branche]"
6,dieser,[]
7,Branche,[dieser]
8,einen,[]
9,Schwerpunkt,[einen]


In [23]:
descendants = []
for token in short_doc:
    descendants.append(
        (
            token.text,
            [descendant.text for descendant in token.subtree if token != descendant],
        )
    )

descendants = pd.DataFrame(descendants, columns=("TOKEN_TEXT", "DESCENDANTS"))
show(descendants)

,TOKEN_TEXT,DESCENDANTS
0,Die,[]
1,Ereignisse,"[Die, in, Japan]"
2,in,[Japan]
3,Japan,[]
4,haben,"[Die, Ereignisse, in, Japan, uns, gezeigt, ,, dass, das, sogenannte, Restrisiko, durchaus, existent, ist, und, dass, es, sich, hierbei, nicht, nur, um, eine, rechnerische, Größe, handelt, .]"
5,uns,[]
6,gezeigt,"[uns, ,, dass, das, sogenannte, Restrisiko, durchaus, existent, ist, und, dass, es, sich, hierbei, nicht, nur, um, eine, rechnerische, Größe, handelt]"
7,",",[]
8,dass,[]
9,das,[]


#### Anchestors per token
This looks at the ancestors of the token and returns any words which archs out to the token.

In [24]:
ancestor = []
for token in short_doc:
    ancestor.append(
        (
            token.text,
            [ancestor.text for ancestor in token.ancestors if token != ancestor],
        )
    )

ancestor = pd.DataFrame(ancestor, columns=("TOKEN_TEXT", "ANCESTORS"))
show(ancestor)

,TOKEN_TEXT,ANCESTORS
0,Die,"[Ereignisse, haben]"
1,Ereignisse,[haben]
2,in,"[Ereignisse, haben]"
3,Japan,"[in, Ereignisse, haben]"
4,haben,[]
5,uns,"[gezeigt, haben]"
6,gezeigt,[haben]
7,",","[gezeigt, haben]"
8,dass,"[ist, gezeigt, haben]"
9,das,"[Restrisiko, ist, gezeigt, haben]"


In [25]:
# the zero index picks the first token head of multiple if the syntactic parsing was erroneous
root = [token for token in short_doc if token.head == token][0]
subject = list(root.lefts)[0]
for descendant in subject.subtree:
    print(descendant.text, [ancestor.text for ancestor in descendant.ancestors])

Die ['Ereignisse', 'haben']
Ereignisse ['haben']
in ['Ereignisse', 'haben']
Japan ['in', 'Ereignisse', 'haben']


In [26]:
subject1 = list(root.lefts)
subject2 = list(root.rights)
print(subject1, "\n", subject2)

[Ereignisse] 
 [gezeigt, .]


## Named-Entity Recognition (NER)

In [27]:
for ent in doc.ents:
    print("{:6} {:50}".format(ent.label_, ent.text))

PER    !                                                 
PER    isl                                               
MISC   Erdbebenkatastrophe in Japan                      
LOC    Japan                                             
MISC   deutschen                                         
LOC    Fukushima                                         
LOC    Reaktorkerne                                      
LOC    Deutschland                                       
LOC    Japan                                             
LOC    Japan                                             
LOC    Deutschland                                       
LOC    Japan                                             
MISC   Atomgesetz                                        
MISC   Zeit des Moratoriums                              
LOC    Bundesumweltminister                              
LOC    Ländern                                           
LOC    Auslegungsgrenzen                                 
LOC    Auslegu

In [28]:
displacy_visual(doc, style="ent")

# Sentiment Analysis with TextBlob
spaCy does not ship with sentiment lexicons, therefore I chose TextBlob to lookup sentiment values for German language. TextBlob itself is another NLP library, the TextBlobDE addition provides the German sentiment lexicon and is very easy to use. At this time, subjectivity scores are not integrated into the lexicon, only polarity scores ranging from -1 to 1.

In [29]:
# To download the language model and nltk corpora for TextBlob, make this cell executable
if 1 == 1:
    os.system("python -m textblob.download_corpora")

from textblob_de import TextBlobDE as TextBlob


The next chunk looks through the entire document already used above and retrieves and lemmatized words used which are assigned either a positive or negative polarity. It's this easy.

In [30]:
# https://spacy.io/usage/linguistic-features#accessing
for token in doc:
    if TextBlob(token.lemma_).sentiment[0] != 0:
        print("{:20} {:}".format(token.lemma_, TextBlob(token.lemma_).sentiment))

geehrt               Sentiment(polarity=1.0, subjectivity=0.0)
lieben               Sentiment(polarity=1.0, subjectivity=0.0)
wirklich             Sentiment(polarity=1.0, subjectivity=0.0)
unzweifelhaft        Sentiment(polarity=0.7, subjectivity=0.0)
groß                 Sentiment(polarity=0.7, subjectivity=0.0)
dramatisch           Sentiment(polarity=0.7, subjectivity=0.0)
unabhängig           Sentiment(polarity=0.7, subjectivity=0.0)
richtig              Sentiment(polarity=1.0, subjectivity=0.0)
richtig              Sentiment(polarity=1.0, subjectivity=0.0)
robust               Sentiment(polarity=0.7, subjectivity=0.0)
zusätzlich           Sentiment(polarity=-0.7, subjectivity=0.0)
groß                 Sentiment(polarity=0.7, subjectivity=0.0)
verheerend           Sentiment(polarity=-1.0, subjectivity=0.0)
natürlich            Sentiment(polarity=0.7, subjectivity=0.0)
natürlich            Sentiment(polarity=0.7, subjectivity=0.0)
vernünftig           Sentiment(polarity=1.0, subjecti